In [41]:
# Importamos las bibliotecas necesarias para el funcionamiento del script.
# `requests`: Se utiliza para realizar solicitudes HTTP y obtener el contenido de páginas web.
# `BeautifulSoup` (importado como `bs`): Es una herramienta poderosa para parsear y manipular documentos HTML/XML.
# `json`: Permite trabajar con datos en formato JSON, como guardar y cargar archivos estructurados.
import requests
from bs4 import BeautifulSoup as bs
import json

In [42]:
# Definimos la función `get_information`, que extrae información de una página wiki específica.
# Argumentos:
# - `full_path`: La URL completa de la página wiki que queremos analizar.
def get_information(full_path):
  try:
    # Realizamos una solicitud GET a la URL proporcionada (`full_path`) usando la biblioteca `requests`.
    r = requests.get(full_path)
    
    # Convertimos el contenido de la respuesta en un objeto BeautifulSoup.
    soup = bs(r.content, 'html.parser')
    
    # Buscamos un elemento con la clase `wikitable infobox`. Este elemento típicamente contiene información estructurada sobre un tema en Wikipedia.
    info_box = soup.find(class_='wikitable infobox')
    if not info_box:
      print(f"No infobox found on page: {full_path}")
      return None  # Devolvemos None si no hay información estructurada
    
    # Extraemos todas las filas (`tr`) dentro del cuadro de información.
    info_rows = info_box.find_all('tr')
    
    # Creamos un diccionario vacío llamado `character_info` para almacenar los datos extraídos.
    character_info = {}
    character_info['Source'] = full_path  # Agregamos la fuente (la URL de la página).
    
    for index, row in enumerate(info_rows):
      if index == 0:
        # La primera fila suele contener el nombre principal del personaje o tema.
        character_info['Name'] = row.find('th').get_text().split('\n')[0]
      
      elif row.find('td') is None:
        # Si la fila no contiene celdas de datos (`td`), usamos el texto del encabezado como clave con valor True.
        content_key = row.find('th').get_text()
        character_info[content_key] = True
      
      else:
        # Procesamos otras filas según su contenido.
        if (len(row.find_all('th')) and len(row.find_all('td'))) > 1:
          # Si la fila contiene múltiples encabezados (`th`) y múltiples celdas de datos (`td`),
          # iteramos sobre ellos y los agregamos al diccionario `character_info`.
          content_keys = row.find_all('th')
          content_values = row.find_all('td')
          for key, value in zip(content_keys, content_values):
            character_info[key.get_text()] = value.get_text()
        
        elif row.find_all('img'):
          # Si la fila contiene imágenes (`img`), tratamos de extraer información relevante de ellas.
          content_key = row.find('th').get_text()
          content_value = ''
          for i, img in enumerate(row.select('img')):
            if i == 1:  # Solo seleccionamos la segunda imagen.
              content_value = img['alt']
          character_info[content_key] = content_value
        
        elif len(row.select('td a')) > 1:
          # Si la fila contiene múltiples enlaces (`a`) dentro de una celda de datos (`td`),
          # concatenamos todos los textos de las celdas separándolos por comas.
          content_key = row.find('th').get_text()
          content_value = row.find('td').get_text().replace('\n', ', ')
          character_info[content_key] = content_value
        
        else:
          # Para todos los demás casos, simplemente extraemos el texto del encabezado (`th`) y el texto de la celda de datos (`td`).
          content_key = row.find('th').get_text()
          content_value = row.find('td').get_text()
          character_info[content_key] = content_value
    
    # Finalmente, devolvemos el diccionario `character_info` con todos los datos extraídos.
    return character_info
  
  except Exception as e:
    # Si ocurre algún error durante el proceso, mostramos un mensaje de advertencia con detalles del error.
    print(f'Error processing page {full_path}: {e}')
    return None

In [43]:
# Definimos la función `save_data`, que guarda los datos en un archivo JSON.
# Argumentos:
# - `title`: El nombre del archivo donde se guardarán los datos.
# - `data`: Los datos que queremos guardar. Estos deben ser serializables a JSON.
def save_data(title, data):
  with open(title, 'w', encoding='utf-8') as f:
    # La función `json.dump()` convierte el objeto Python `data` en una cadena JSON y lo guarda en el archivo `f`.
    json.dump(data, f, ensure_ascii=False, indent=2)

# Definimos la función `load_data`, que carga datos desde un archivo JSON.
# Argumentos:
# - `title`: El nombre del archivo JSON desde donde se cargarán los datos.
def load_data(title):
  with open(title, encoding='utf-8') as f:
    # La función `json.load()` lee el contenido del archivo `f` y lo convierte en un objeto Python (por ejemplo, una lista o diccionario).
    return json.load(f)

In [44]:
# Definimos la URL base del sitio web. Esta será usada para construir las URLs completas a partir de enlaces relativos.
# La URL base es comúnmente utilizada cuando los enlaces en una página son relativos (es decir, no incluyen el dominio completo).
base_link = 'https://en.uesp.net'

# Realizamos una solicitud GET a la URL proporcionada ('https://en.uesp.net/wiki/Skyrim:People').
# El objeto `r` contiene la respuesta de la solicitud, incluyendo el contenido HTML de la página.
# En este caso, estamos accediendo a una página que contiene información sobre los personajes del juego "Skyrim".
# Ejemplo: Esta línea obtendrá todo el contenido HTML de la página donde se listan las personas en Skyrim.
r = requests.get('https://en.uesp.net/wiki/Skyrim:People')

# Convertimos el contenido de la respuesta en un objeto BeautifulSoup.
# Esto nos permite trabajar con el HTML de manera más estructurada y fácil de manipular.
# Usamos 'html.parser' como motor de análisis, que es un parser incluido con Python por defecto.
# Este paso es crucial porque convierte el contenido HTML en un formato que podemos navegar y buscar fácilmente.
soup = bs(r.content, 'html.parser')

# Utilizamos el método `select` de BeautifulSoup para encontrar todos los elementos que cumplen con el selector CSS especificado.
# `.wikitable tr b > a`: Este selector busca dentro de una tabla con la clase `wikitable`, todas las filas (`tr`), luego dentro de cada fila busca etiquetas `<b>` y finalmente selecciona los enlaces (`<a>`) directamente dentro de estas etiquetas `<b>`.
# Este selector específico está diseñado para extraer los enlaces de los nombres de los personajes en la tabla de personas de Skyrim.
# Ejemplo: Si hay una fila con `<tr><td><b><a href="/wiki/Character1">Character1</a></b></td></tr>`, este selector capturará el enlace `/wiki/Character1`.
people_list = soup.select('.wikitable tr b > a')

# Explicación detallada de cómo funciona el selector CSS:
# - `.wikitable`: Busca una tabla con la clase `wikitable`. Es común que las tablas en sitios wiki tengan esta clase.
# - `tr`: Dentro de esa tabla, busca todas las filas (`tr`).
# - `b > a`: Dentro de cada fila, busca etiquetas `<b>` que contienen directamente un enlace `<a>`.
# Este selector asegura que solo obtenemos los enlaces relacionados con los nombres de los personajes, ignorando otros enlaces o elementos irrelevantes.

# Resultado esperado:
# La variable `people_list` ahora contiene una lista de objetos `Tag` de BeautifulSoup, donde cada objeto representa un enlace (`<a>`) encontrado.
# Cada elemento de esta lista puede ser usado para extraer atributos como `href` (la URL del enlace) o el texto visible del enlace.
# Ejemplo: Si `people_list[0]` es `<a href="/wiki/Character1">Character1</a>`, entonces:
# - `people_list[0].get('href')` devolverá "/wiki/Character1".
# - `people_list[0].get_text()` devolverá "Character1".

# Nota: Para ejecutar este código correctamente, asegúrate de tener instaladas las bibliotecas `requests` y `beautifulsoup4`.
# Puedes instalarlas usando pip:
# pip install requests beautifulsoup4

In [45]:
# Scrape the data based on the links
# Iniciamos una lista vacía llamada `characters_data` que almacenará los datos de cada personaje obtenidos del scraping.
# Esta lista actuará como un contenedor para todos los diccionarios con información de los personajes.
characters_data = []

# Iteramos sobre la lista `people_list`, que se supone que contiene objetos `Tag` de BeautifulSoup representando enlaces a páginas de personajes.
# Usamos `enumerate` para tener acceso al índice (`index`) y al elemento actual (`people`) en cada iteración.
# Ejemplo: Si `people_list` contiene [<a href="/wiki/Character1">Character1</a>, <a href="/wiki/Character2">Character2</a>], 
# entonces en la primera iteración `people` sería `<a href="/wiki/Character1">Character1</a>` y `index` sería 0.
for index, people in enumerate(people_list):
  try:
    # Obtenemos el enlace relativo usando el atributo `href` del objeto `people`.
    # Usamos `.get('href', '')` para evitar errores si el atributo `href` no existe.
    relative_link = people.get('href', '')
    
    # Si el enlace relativo está vacío, lo omitimos e imprimimos un mensaje de advertencia.
    if not relative_link:
      print(f"Skipping empty link at index {index}")
      continue
    
    # Construimos la URL completa concatenando la URL base (`base_link`) con el enlace relativo (`relative_link`).
    fullpath = base_link + relative_link
    
    # Filtramos enlaces no relevantes (categorías o listas).
    # Si el texto del enlace contiene palabras clave como "Followers", "Merchants" o "Trainers", lo omitimos.
    # Filtramos URLs específicas que no queremos procesar.
    if any(forbidden_url in fullpath for forbidden_url in ['Dark_Brotherhood_Initiate', 'Followers', 'Merchants', 'Trainers']):
      print(f"Skipping forbidden URL: {fullpath}")
      continue
    
    # Verificamos si el enlace apunta a una redirección o página inválida.
    # Si el enlace contiene 'redlink=1' o está vacío después de eliminar espacios, lo omitimos.
    if 'redlink=1' in fullpath or not relative_link.strip():
      print(f"Skipping invalid link: {fullpath}")
      continue
    
    # Procesamos solo enlaces válidos llamando a la función `get_information`.
    data = get_information(fullpath)
    
    # Si `get_information` devuelve datos válidos, los añadimos a la lista `characters_data`.
    if data:
      characters_data.append(data)
  
  except Exception as e:
    # Si ocurre algún error durante el procesamiento de un enlace, mostramos un mensaje de advertencia con detalles del error.
    print(f'WARNING: Failed to process {people.get_text()} {fullpath} - Error: {e}')


Skipping forbidden URL: https://en.uesp.net/wiki/Skyrim:Followers
Skipping forbidden URL: https://en.uesp.net/wiki/Skyrim:Merchants
Skipping forbidden URL: https://en.uesp.net/wiki/Skyrim:Trainers
Skipping forbidden URL: https://en.uesp.net/wiki/Skyrim:Dark_Brotherhood_Initiate
Skipping forbidden URL: https://en.uesp.net/wiki/Skyrim:Followers
No infobox found on page: https://en.uesp.net/wiki/Skyrim:Night_Mother
No infobox found on page: https://en.uesp.net/wiki/Skyrim:Night_Mother
Skipping forbidden URL: https://en.uesp.net/wiki/Skyrim:Followers
No infobox found on page: https://en.uesp.net/wiki/Skyrim:Orc
No infobox found on page: https://en.uesp.net/wiki/Skyrim:Armored_Troll
Skipping forbidden URL: https://en.uesp.net/wiki/Skyrim:Followers
Skipping invalid link: https://en.uesp.net/w/index.php?title=Skyrim:Imperial_Soldier_(Sovngarde)&action=edit&redlink=1


In [46]:
# Guardamos los datos recopilados previamente (`characters_data`) en un archivo llamado 'skyrim_population.json'.
# La función `save_data` toma el nombre del archivo ('skyrim_population.json') y los datos (`characters_data`) como argumentos.
# Ejemplo: Si `characters_data` contiene [{'Name': 'Alduin', 'Race': 'Dragon'}, {'Name': 'Ulfric', 'Race': 'Human'}],
# este código creará un archivo JSON con estos datos estructurados.
save_data('skyrim_population.json', characters_data)

# Explicación adicional:
# - La función `save_data` es útil para almacenar grandes volúmenes de datos estructurados en un archivo persistente.
# - La función `load_data` permite recuperar esos datos más tarde sin necesidad de volver a realizar el scraping.
# - Ambas funciones utilizan la codificación UTF-8 para garantizar que todos los caracteres, incluso aquellos fuera del alfabeto inglés, sean manejados correctamente.

# Ejemplo de uso de `load_data`:
# Supongamos que hemos guardado los datos anteriormente y ahora queremos cargarlos:
# loaded_data = load_data('skyrim_population.json')
# print(loaded_data)  # Esto imprimirá los datos cargados desde el archivo JSON.
# print({len(loaded_data)})  # Esto imprimirá el numero de datos cargados desde el archivo JSON.
print(f"Total characters processed: {len(characters_data)}")

Total characters processed: 1062


In [47]:
# Definimos la función `extract_image_url`, que extrae la URL de la imagen desde una página wiki.
# Argumentos:
# - `full_path`: La URL completa de la página wiki que queremos analizar.
def extract_image_url(full_path):
  try:
    # Realizamos una solicitud GET a la URL proporcionada (`full_path`) usando la biblioteca `requests`.
    r = requests.get(full_path)
    
    # Convertimos el contenido de la respuesta en un objeto BeautifulSoup.
    soup = bs(r.content, 'html.parser')
    
    # Buscamos el elemento <meta property="og:image"> que contiene la URL de la imagen.
    meta_tag = soup.find('meta', property='og:image')
    
    # Si encontramos el elemento, devolvemos su atributo 'content' (la URL de la imagen).
    if meta_tag and 'content' in meta_tag.attrs:
      return meta_tag['content']
    
    # Si no se encuentra ninguna imagen, mostramos un mensaje de advertencia y devolvemos `None`.
    print(f"No image found for page: {full_path}")
    return None
  
  except Exception as e:
    # Si ocurre algún error durante el procesamiento, mostramos un mensaje de advertencia con detalles del error.
    print(f"Error extracting image from {full_path}: {e}")
    return None

In [48]:
# Definimos la función `save_data` que se encarga de guardar datos en un archivo JSON.
# Argumentos:
# - `title`: El nombre del archivo donde se guardarán los datos (por ejemplo, 'skyrim_population.json').
# - `data`: Los datos que queremos guardar. Estos deben ser serializables a JSON (por ejemplo, listas, diccionarios, etc.).
def save_data(title, data):
  # Usamos el contexto `with open()` para abrir el archivo especificado por `title` en modo escritura (`'w'`).
  # El argumento `encoding='utf-8'` asegura que los caracteres especiales (como tildes o letras no latinas) sean correctamente guardados.
  with open(title, 'w', encoding='utf-8') as f:
    # La función `json.dump()` convierte el objeto Python `data` en una cadena JSON y lo guarda en el archivo `f`.
    # - `ensure_ascii=False`: Permite que los caracteres no ASCII (como tildes o acentos) sean almacenados directamente, en lugar de ser escapados.
    # - `indent=2`: Formatea el archivo JSON con sangría de 2 espacios para mejorar su legibilidad.
    json.dump(data, f, ensure_ascii=False, indent=2)

In [49]:
# Cargamos los datos previamente extraídos desde el archivo `skyrim_population.json`.
try:
  with open('skyrim_population.json', encoding='utf-8') as f:
    characters_data = json.load(f)  # Cargamos los datos como una lista de diccionarios.
except FileNotFoundError:
  print("El archivo skyrim_population.json no fue encontrado.")
  characters_data = []  # Si el archivo no existe, inicializamos una lista vacía.

In [50]:
# Creamos una lista vacía llamada `characters_images` para almacenar los nuevos datos (Source, Name e Image).
characters_images = []

# Procesamos cada entrada en `characters_data`.
for character in characters_data:
  try:
    # Obtenemos la URL del personaje (clave 'Source') y su nombre (clave 'Name').
    source_url = character.get('Source')
    name = character.get('Name', 'Unknown')  # Usamos 'Unknown' si no hay nombre.
    
    # Verificamos si la URL existe.
    if not source_url:
      print(f"Skipping entry without a valid source URL for character: {name}")
      continue
    
    # Extraemos la URL de la imagen usando la función `extract_image_url`.
    image_url = extract_image_url(source_url)
    
    # Si se encontró una imagen, la añadimos a la lista junto con la URL del personaje y su nombre.
    if image_url:
      characters_images.append({
        'Source': source_url,
        'Name': name,
        'Image': image_url
      })
  
  except Exception as e:
    # Si ocurre algún error durante el procesamiento, mostramos un mensaje de advertencia con detalles del error.
    print(f"Error processing character {character.get('Name', 'Unknown')} - Source: {source_url} - Error: {e}")

In [51]:
# Guardamos los datos resultantes en un nuevo archivo JSON llamado `skyrim_images.json`.
save_data('skyrim_images.json', characters_images)

# Mostramos el número total de entradas procesadas correctamente.
print(f"Total characters-images processed: {len(characters_images)}")

Total characters-images processed: 1062
